# 책 추천 시스템

In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

### 0. 데이터

책 데이터

In [ ]:
books = pd.read_csv('./books.csv', encoding='ISO-8859-1')
print(books.shape)
books.head(2)

평점 데이터

In [ ]:
ratings = pd.read_csv('./ratings.csv', encoding='ISO-8859-1')
print(ratings.shape)
ratings.head(2)

책 태그

In [ ]:
book_tags = pd.read_csv('./book_tags.csv', encoding='ISO-8859-1')
print(book_tags.shape)
book_tags.head(2)

태그 정보

In [ ]:
tags = pd.read_csv('./tags.csv')
print(tags.shape)
tags.tail(2)

책 태그와 태그정보 merge

In [ ]:
tags_join_df = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
tags_join_df.head(2)

유저가 읽은 책

In [ ]:
to_read = pd.read_csv('./to_read.csv')
print(to_read.shape)
to_read.head(2)

TF-IDF Vectorize
- authors로 Tfidf 수행

In [ ]:
books['authors'][:5]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(books['authors'])
tfidf_matrix

linear_kernel을 통한 유사도 측정
- linear_kernel : 코사인 유사도를 구하는 다른 방법

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

authors 기준 유사한 책 찾기

원하는 책의 인덱스 찾기

In [ ]:
titles = books['title']
indices = pd.Series(books.index, index=books['title'])
indices['The Hobbit']

해당 책의 유사도 값 호출

In [ ]:
cosine_sim[indices['The Hobbit']]

가장 유사한 책의 인덱스 찾기

In [ ]:
# 유사도 결과 인덱스를 가진 list형으로 변형
sim_scores = list(enumerate(cosine_sim[indices['The Hobbit']]))
sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
sim_scores[:3]

작가로 유사한 책 찾기

In [ ]:
sim_scores_10 = sim_scores[1: 11]
book_indices = [i[0] for i in sim_scores_10]
titles.iloc[book_indices]

Tag 추가

In [ ]:
books_with_tags = pd.merge(books, tags_join_df, left_on='book_id', right_on='goodreads_book_id', how='inner')
books_with_tags.head(2)

TfidfVectorize
- tag

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
tfidf_matrix1

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
cosine_sim1

tag 기준 유사한 책 찾기

추천책 반환 함수

In [ ]:
titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

def tags_recommendations(title, top_n=10):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim1[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]

    return titles1.iloc[book_indices]

tags_recommendations('The Hobbit', 15)

corpus(author + Tag) 추가

각 영화의 태그를 합쳐 문자열로 구성

In [ ]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head(2)

books에 merge

In [ ]:
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='inner')
books.head(2)

저자 이름과 태그 합치기

In [ ]:
books['corpus'] = (pd.Series(books[['authors', 'tag_name']].fillna('').values.tolist())).str.join(' ')
books['corpus'][:3]

TfidfVectorize
- corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_corpus = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])
cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)

titles = books['title']
indices = pd.Series(books.index, index=books['title'])

In [ ]:
def corpus_recommendations(title, top_n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_corpus[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]

    return titles.iloc[book_indices]

In [ ]:
corpus_recommendations('The Hobbit', 15)

In [ ]:
corpus_recommendations('Twilight (Twilight, #1)', 15)